Seed for prng2 of 11 + 12 doesn't really converge, 13 does! RP trick tends to yield less noisy gradients but convergence slower.

Convergence depends on initial conditions

To do: Natural gradient, write up commentary, benchmarks, cython

Large parameters cause gradients to be too large!

In [14]:
import numpy as np
from numpy.random import RandomState
import matplotlib.pyplot as plt
import scipy.stats as stats
from mpl_toolkits.mplot3d import Axes3D
import scipy.special as s_spec
import scipy
from numpy import sqrt
from numpy import exp
from numpy import log

###############generate simulated dataset from Poisson distribution###############

#set parameters 
lambda_data =  5
n = 20 #no. of samplest

#generate dataset
prng1 = RandomState(0)
data = prng1.poisson(lambda_data, n)
x_bar = np.mean(data)
s = np.var(data) #variance is divided by n not n-1

#set prior distribution parameters
a_0 = 4
b_0 = 0.7

#initialise initial variational parameter estimates
prng2 = RandomState(13)

#set parameters for MC gradient evaluation
n_samples = 100
learn = 0.001
n_iter = 7000 #iterations for gradient ascent
#set generalised gradient parameters
n_samples_grg = 100

We now characterise the posterior analytically. First, we assume there are $n$ i.i.d observations and they follow a Poisson distribution so

$L(\textbf{x} | \lambda) = \prod_{i=1}^{n} \frac{\lambda^{x_i}e^\lambda}{x_i!}$ 

Next, we assume the $\lambda$ has a gamma prior distribution 

$\lambda \sim Gamma(\alpha_0,\beta_0)$

where for $X\sim Gamma(\alpha,\beta)$, we use the Gamma density of the form $f(x|\alpha,\beta)=\frac{\beta^\alpha}{\Gamma{(\alpha})}x^{\alpha-1}e^{\beta x}$. Given this likelihood and prior, the posterior is also a gamma distribution

$\lambda|\textbf{x}\sim Gamma(\alpha_0+\sum_{i=1}^n x_i,\beta_0+n)$


In [2]:
###############Calculate analytically posterior parameters###############

#see above for formula
a_pos = a_0+x_bar*n
b_pos = b_0+n

print('The posterior parameters are: (alpha = {:0.2f}, beta = {:0.2f})'.format(a_pos, b_pos))

The posterior parameters are: (alpha = 100.00, beta = 20.70)


We assume the form of the VB approximation of the posterior is 

$q(\textbf{z};\textbf{v})=\frac{{\beta_n}^{\alpha_n}}{\Gamma{(\alpha_n})}\lambda^{\alpha_n-1}e^{\beta_n \lambda}$

We seek to find the optimal value of the parameter vector $\textbf{v}=[\alpha_n,\beta_n]$ by minimising the KL divergence between $q$ and the actual posterior. In this case, the parameters should match exactly since the posterior is also a gamma distribution.

To find the optimal values for the parameters, we seek to maximise the variational lower bound $\mathcal{L}(\textbf{v})$ w.r.t. $\textbf{v}$ where 

$\mathcal{L}(\textbf{v}) = [\log p(\textbf{x},\textbf{z}) - \log q(\textbf{z};\textbf{v})] = \mathbb{E}_{q(\textbf{z},\textbf{v})}[f(\textbf{z})]+\mathbb{H}_{q(\textbf{z},\textbf{v})}[q(\textbf{z},\textbf{v})]$

where $\textbf{x}$ is the data, $\textbf{z}=\lambda$, and $\mathbb{H}_{q(\textbf{z},\textbf{v})}[q(\textbf{z},\textbf{v})]$ is the entropy of $q(\textbf{z},\textbf{v})$.

and $f(\textbf{z})=\log p(\textbf{x},\textbf{z})=\sum_{i=1}^n \log p(x_i|\textbf{z}) + \log p(\textbf{z};\textbf{v})$

We use some form of gradient ascent to maximise the lower bound, and from the literature there are two different ways to do this. The first way is called the *score function* method which utilises the log-derivative trick to estimate the gradient. This method supposedly has high variance and is in general not able to reliably estimate the gradient. The second way is using the *generalised reparameterisation gradient* outlined in the paper by Ruiz et. al. (2016). 

To estimate $\nabla_{\textbf{v}}\mathcal{L}$ using the score function method, we seek to evaluate

$\nabla_{\textbf{v}}\mathcal{L}=\nabla_{\textbf{v}}(\mathbb{E}_{q(\textbf{z},\textbf{v})}[f(\textbf{z})]+\mathbb{H}_{q(\textbf{z},\textbf{v})}[q(\textbf{z},\textbf{v})])=\nabla_{\textbf{v}}\mathbb{E}_{q(\textbf{z}
,\textbf{v})}[f(\textbf{z})]+\nabla_{\textbf{v}}\mathbb{H}_{q(\textbf{z},\textbf{v})}[q(\textbf{z},\textbf{v})]$

We have that $\nabla_{\textbf{v}}\mathbb{E}_{q(\textbf{z},\textbf{v})}[f(\textbf{z})]=\nabla_{\textbf{v}}\int q(\textbf{z},\textbf{v})f(\textbf{z}) dz=\int \nabla_{\textbf{v}} q(\textbf{z},\textbf{v})f(\textbf{z}) dz=\int \frac{q(\textbf{z},\textbf{v})}{q(\textbf{z},\textbf{v})}\nabla_{\textbf{v}} q(\textbf{z},\textbf{v})f(\textbf{z}) dz=\int q(\textbf{z},\textbf{v})f(\textbf{z})\nabla_{\textbf{v}}\log q(\textbf{z},\textbf{v}) = \mathbb{E}_{q(\textbf{z},\textbf{v})}[f(\textbf{z})\nabla_{\textbf{v}}\log q(\textbf{z},\textbf{v})]$

We now need to estimate the final term above and the gradient of the entropy. First, given our choices of likelihood, prior and $q$, we have that

$f(\textbf{z})=\log p(\textbf{x},\textbf{z}) = \alpha_0 \log{\beta_0}-\log{\Gamma(\alpha_0)}-\sum_{i=1}^n x_i! + (\sum_{i=1}^n x_i + \alpha_0 - 1)\log{\textbf{z}}-(\beta_0+n)\textbf{z}$ 

Also, $\nabla_{\textbf{v}}\log q(\textbf{z},\textbf{v})=[\frac{\partial}{\partial\alpha_n}\log q(\textbf{z},\textbf{v}), \frac{\partial}{\partial\beta_n}\log q(\textbf{z},\textbf{v})]^{\intercal}$ where

$\frac{\partial}{\partial\alpha_n}\log q(\textbf{z},\textbf{v}) = \log \beta_n - \psi(\alpha_n)+\log(\textbf{z})$ where $\psi$ is the digamma function.

$\frac{\partial}{\partial\beta_n}\log q(\textbf{z},\textbf{v})=\frac{\alpha_n}{\beta_n}-\textbf{z}$

Finally, the entropy can be shown to be

$\mathbb{H}_{q(\textbf{z},\textbf{v})}[q(\textbf{z},\textbf{v})]=\alpha_n-\log \beta_n + \log \Gamma(\alpha_n) + (1-\alpha_n)\psi(\alpha_n)$

and so we have that the gradient of the entropy is 

$\nabla_{\textbf{v}}\mathbb{H}_{q(\textbf{z},\textbf{v})}[q(\textbf{z},\textbf{v})]=[\frac{\partial}{\partial\alpha_n}\mathbb{H}_{q(\textbf{z},\textbf{v})},\frac{\partial}{\partial\beta_n}\mathbb{H}_{q(\textbf{z},\textbf{v})}]^\intercal$ where

$\frac{\partial}{\partial\alpha_n}\mathbb{H}_{q(\textbf{z},\textbf{v})}=1-\alpha_n \psi_1(\alpha_n)$ where $\psi_n$ is the $n^{th}$ derivative of the digamma function and

$\frac{\partial}{\partial\beta_n}\mathbb{H}_{q(\textbf{z},\textbf{v})} = -\frac{1}{\beta_n}$


In [15]:
def f(z):
    return a_0*np.log(b_0)-np.log(s_spec.digamma(a_0))-np.sum(np.log(scipy.misc.factorial(data)))+ \
(n*x_bar+a_0-1)*np.log(z)-(b_0+n)*z

#transforms variational parameters onto the real line
def T_v(xi):
    v = exp(xi)
    return v

def T_inv_v(v):
    xi = log(v)
    return v

def T_v_grad(xi):
    v_g = exp(xi)
    return v_g

#calculate elements of the ELBO gradient

def entropy_grad(xi):
    grad = np.empty(2)
    #retrieve parameters
    v = T_v(xi)
    a = v[0]
    b = v[1]
    #retrieve gradient of transform
    v_g = T_v_grad(xi)
    print('v_g',v_g)
    #calculate partial derivatives of entropy
    grad[0] = (1 + (1-a)*s_spec.polygamma(1,a))*v_g[0]
    grad[1] = -1/b*v_g[1]
    return grad

def ln_q_deriv_xi(z,xi):
    #retrieve parameters
    v = T_v(xi)
    a = v[0]
    b = v[1]
    #retrieve gradient of transform
    v_g = T_v_grad(xi)
    #evaluate final gradient
    d_ln_q = np.empty([2,z.size])
    d_ln_q[0] = (np.log(b)-s_spec.digamma(a)+np.log(z))*v_g[0]
    d_ln_q[1] = (a/b-z)*v_g[1]
    return d_ln_q

def lb_grad_score(z, xi):
    grad = np.empty([2,z.size])
    #retrieve parameters
    v = T_v(xi)
    a_n = v[0]
    b_n = v[1]
    #calculate partial derivatives of log(q(z;v))
    q_del = ln_q_deriv_xi(z,v) 
    #set gradient
    grad[0] = f(z)*q_del[0]
    grad[1] = f(z)*q_del[1]
    return grad

def ELBO(xi):
    #retrieve parameters
    v = T_v(xi)
    a_n = v[0]
    b_n = v[1]
    #evaluate expectation of f(z) under q (feed expectation through z terms)
    E_f = a_0*np.log(b_0)-np.log(s_spec.digamma(a_0))-np.sum(np.log(scipy.misc.factorial(data)))+ \
    (n*x_bar+a_0-1)*(s_spec.digamma(a_n) - np.log(b_n))-(b_0+n)*a_n/b_n
    entropy = a_n-np.log(b_n)+np.log(s_spec.gamma(a_n))+(1-a_n)*s_spec.digamma(a_n)
    cost = E_f + entropy
    return cost

In [13]:
###############estimate gradient using MC###############

#initialise parameters randomly
xi_0 = prng2.uniform(0,1,2)
xi = xi_0
#convert using transformation to parameter space
v = T_v(xi)
#print initialisation of parameter
print("Initialised parameters: ",v)
#initialise graph vectors
cost_graph = np.empty(n_iter)
grad_var = np.empty([n_iter,2])

for i in range(0,n_iter):
    #print(i)
    print('v',v)
    samples = np.random.gamma(v[0],1/v[1],n_samples)
    grad_vect = lb_grad_score(samples, xi) #for variance
    grad = np.mean(grad_vect,axis=1) + entropy_grad(xi)
    #update variance of gradient for comparison to RP
    grad_var[i] = np.var(grad_vect, axis=1)
    #update transformed variational parameters
    print('grad',grad)
    xi = xi + learn*grad
    #convert back to parameter space
    v = T_v(xi)
    #record the cost for plotting to check convergence
    c = ELBO(xi)
    cost_graph[i]= c
    
print("Final variational parameters: ",v)

#check convergence

plt.plot(np.linspace(1,n_iter,n_iter), cost_graph, label='Score function method')
plt.ylabel('ELBO')
plt.xlabel('n')
plt.title('Convergence of ELBO')
plt.show()

Initialised parameters:  [ 1.2921144   1.41563928]
v [ 1.2921144   1.41563928]
v_g [ 1.2921144   1.41563928]
grad [ 461.62641218 -177.0030442 ]
v [ 130.65288467    0.24112273]
v_g [ 130.65288467    0.24112273]
grad [  7.29490243e+62  -5.87745717e+60]
v [ inf   0.]
v_g [ inf   0.]
grad [ nan  nan]
v [ nan  nan]


/Users/mingxu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: RuntimeWarning: overflow encountered in exp
/Users/mingxu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:65: RuntimeWarning: divide by zero encountered in log
/Users/mingxu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:65: RuntimeWarning: invalid value encountered in double_scalars
/Users/mingxu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:66: RuntimeWarning: divide by zero encountered in log
/Users/mingxu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:66: RuntimeWarning: invalid value encountered in double_scalars
/Users/mingxu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:17: RuntimeWarning: divide by zero encountered in double_scalars
/Users/mingxu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:42: RuntimeWarning: invalid value encountered in add
/Users/mingxu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:43: RuntimeWarn

ValueError: shape < 0

In [ ]:
#summary metrics - compare to analytical result
print('Converged ELBO: ',c) 
print('ELBO of analytical posterior parameters: ', ELBO(T_inv_v([a_pos,b_pos])))
print('Mean of VB approximation',v[0]/v[1])
print('Mean of analytic posterior: ',a_pos/b_pos)

#set up plot for pdf of posterior under VB
vb_mean = v[0]/v[1]
vb_sd = vb_mean/v[1]
x = np.linspace(vb_mean-8*vb_sd, vb_mean+8*vb_sd, 200) #generate sample space for plotting density
pdf_vb = stats.gamma.pdf(x, v[0], scale=1/v[1])

#set up plot for pdf of analytical posterior
a_mean = a_pos/b_pos
a_sd = a_mean/b_pos
#x = np.linspace(a_mean-4*a_sd, a_mean+4*a_sd, 200)
pdf_a = stats.gamma.pdf(x, a_pos, scale=1/b_pos)

#conditional/marginal for tau
plt.plot(x, pdf_a, 'b', label='Analytical')
plt.plot(x, pdf_vb, 'r', label='VB approximation')
plt.ylabel('p(x)')
plt.xlabel('x')
plt.legend()
plt.title(r'$\tau$')
plt.show()

Now we use the generalised reparameterisation gradient by Ruiz et. al. to try and find a low variance estimate of the gradient.


In [ ]:
def T(e, xi):
    #retrieve parameters
    v = T_v(xi)
    a_n = v[0]
    b_n = v[1]
    return np.exp(e*np.sqrt(s_spec.polygamma(1,a_n))+s_spec.digamma(a_n)-np.log(b_n))

def Tinv(z, xi):
    #retrieve parameters
    v = T_v(xi)
    a_n = v[0]
    b_n = v[1]
    return (np.log(z)-s_spec.digamma(a_n) + np.log(b_n))/np.sqrt(s_spec.polygamma(1,a_n))

def h(e, xi):
    #retrieve parameters
    v = T_v(xi)
    a = v[0]
    b = v[1]
    h = np.empty([2,e.size])
    pg_1 = s_spec.polygamma(1,a)
    #bring in gradient of T_v
    v_g = T_v_grad(xi)
    #calculate h
    h[0] = T(e, v)*(e*s_spec.polygamma(2,a)/(2*np.sqrt(pg_1))+ pg_1)*v_g[0]
    h[1] = -T(e, v)/b*v_g[1]
    return h

def u(e, v):
    #retrieve parameters
    v = T_v(xi)
    a = v[0]
    b = v[1]
    #bring in gradient of T_v
    v_g = T_v_grad(xi)
    #set up output for u
    u = np.empty([2,e.size])
    pg_1 = s_spec.polygamma(1,a)
    pg_2 = s_spec.polygamma(2,a)
    u[0] = (e*pg_2/(2*np.sqrt(pg_1))+ pg_1 + pg_2/(2*pg_1))*v_g[0]
    u[1] = -1/b*v_g[1]
    return u

#calculate Fisher information of variational distribution for natural gradient
def I(xi):
    #find information w.r.t. v
    v=T_v(xi)
    a = v[0]
    b = v[1]
    I_v = np.zeros([2,2])
    I_v[0,0] = -s_spec.polygamma(1,a); I_v[1,1]=-a/b**2
    I_v[0,1]=1/b; I_v[1,0]=1/b
    #evaluate Jacobian of transformation
    J = T_v_J(xi)
    return np.matmul(np.matmul(np.transpose(J),I_v),J)

#evaluate Jacobian of transformation of variation parameters for natural gradient

def T_v_J(xi):
    v_g = exp(xi)
    #initalise Jacobian
    J = np.zeros([2,2])
    J[0,0]=v_g[0]
    J[1,1]=v_g[1]
    return J

def f_deriv(z):
    return (x_bar*n+a_0-1)/z-(b_0+n)

def ln_q_deriv_z(z,v):
    a = v[0]
    b = v[1]
    return (a-1)/z-b

def g_corr(z, v):
    g_corr = np.empty([2,z.size])
    g_corr = f(z)*(ln_q_deriv_z(z,v)*h(Tinv(z,v),v)+ln_q_deriv_xi(z,v)+u(Tinv(z,v),v))
    return g_corr

In [ ]:
#initalise parameters consistent with score method
xi = xi_0
v = T_v(xi)
print("Initialised parameters: ",v)
#initialise graphing arrays
cost_graph1 = np.empty(n_iter)
grad_var1 = np.empty([n_iter,2])
grad_mean1 = np.empty([n_iter,2])
grad_cv1 = np.empty([n_iter,2])

for i in range(0,n_iter):
    samples = np.random.gamma(v[0],1/v[1],n_samples_grg)
    g_rep = f_deriv(samples)*h(Tinv(samples,xi),xi)
    g_cor = g_corr(samples,xi)
    #calculate gradient
    grad_vect = g_rep + g_cor
    grad_rp = np.mean(grad_vect) + entropy_grad(xi)
    #store variance of gradient to compare to score
    grad_var1[i] = np.var(grad_vect, axis=1)
    #update transformed variational parameters
    xi = xi + learn*grad_rp
    v = T_v(xi)
    #record the cost for plotting to check convergence
    c = ELBO(xi)
    cost_graph1[i]= c

print("Final variational parameters: ",v)

#check convergence of ELBO
plt.plot(np.linspace(1,n_iter,n_iter), cost_graph, label='Score function method')
plt.plot(np.linspace(1,n_iter,n_iter), cost_graph1, label='Reparameterisation trick')
plt.ylabel('ELBO')
plt.xlabel('n')
plt.legend()
plt.title('Convergence of ELBO')
plt.show()

In [ ]:
#analysis of gradient variance graphs
plt.title("Variance of gradient by iteration - reparameterisation")
plt.hist(grad_var1,bins=50)
plt.show()

plt.title("Variance of gradient by iteration - score")
plt.hist(grad_var,bins=50)
plt.show()

In [ ]:
#summary metrics - compare to analytical result
print('Converged ELBO: ',c) 
print('ELBO of analytical posterior parameters: ', ELBO(T_inv_v([a_pos,b_pos])))
print('Mean of VB approximation',v[0]/v[1])
print('Mean of analytic posterior: ',a_pos/b_pos)

#set up plot for pdf of posterior under VB
vb_rp_mean = v[0]/v[1]
vb_rp_sd = vb_mean/v[1]
#x = np.linspace(0,150,200)
x = np.linspace(vb_mean-10*vb_sd, vb_mean+10*vb_sd, 200) #generate sample space for plotting density
pdf_vb_rp = stats.gamma.pdf(x, v[0], scale=1/v[1])

#set up plot for pdf of analytical posterior
#x = np.linspace(a_mean-4*a_sd, a_mean+4*a_sd, 200)
pdf_a = stats.gamma.pdf(x, a_pos, scale=1/b_pos)

#conditional/marginal for tau
plt.plot(x, pdf_a, 'b', label='Analytical')
plt.plot(x, pdf_vb_rp, 'r', label='VB approximation')
plt.ylabel('p(x)')
plt.xlabel('x')
plt.legend()
plt.title(r'$\tau$')
plt.show()

We now implement the reparameterisation trick using the natural gradient instead of the standard Euclidean gradient.

In [ ]:
#initalise parameters consistent with score method
xi = xi_0
v = T_v(xi)
print("Initialised parameters: ",v)
#initialise graphing arrays
cost_graph2 = np.empty(n_iter)
grad_var1 = np.empty([n_iter,2])

for i in range(0,5):
    samples = np.random.gamma(v[0],1/v[1],n_samples_grg)
    g_rep = f_deriv(samples)*h(Tinv(samples,xi),xi)
    g_cor = g_corr(samples,xi)
    #calculate gradient
    grad_vect = g_rep + g_cor
    grad_rp = np.mean(grad_vect) + entropy_grad(xi)
    #store variance of gradient to compare to score
    grad_var1[i] = np.var(grad_vect, axis=1)
    #calculate Fisher information matrix for natural gradient
    xi_I = I(xi)
    print(xi_I,np.linalg.inv(xi_I),T_v_J(xi), xi)
    #update transformed variational parameters
    xi = xi + learn*np.matmul(np.linalg.inv(xi_I),grad_rp)
    v = T_v(xi)
    #record the cost for plotting to check convergence
    c = ELBO(xi)
    cost_graph2[i]= c

print("Final variational parameters: ",v)

#check convergence of ELBO
plt.plot(np.linspace(1,n_iter,n_iter), cost_graph, label='Score method')
plt.plot(np.linspace(1,n_iter,n_iter), cost_graph1, label='RP trick')
plt.plot(np.linspace(1,n_iter,n_iter), cost_graph2, label='RP trick w/Natural gradient')
plt.ylabel('ELBO')
plt.xlabel('n')
plt.legend()
plt.title('Convergence of ELBO')
plt.show()